In [5]:
import numpy as np
import pandas as pd
import gc
import random
import string
import os

import torch
import gc
gc.collect()
torch.cuda.empty_cache()
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split

import fasttext
import fasttext.util

from transformers import BertForSequenceClassification, AdamW,BertTokenizer,get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split

### Veri Okuma

In [2]:
halk_dataset = pd.DataFrame()
sanat_dataset = pd.DataFrame()
ilahi_dataset = pd.DataFrame()
pop_dataset = pd.DataFrame()
rock_dataset = pd.DataFrame()
rap_dataset = pd.DataFrame()

halk_dataset["label"] = [0 for _ in range(200)]
sanat_dataset["label"] = [1 for _ in range(200)]
ilahi_dataset["label"] = [2 for _ in range(200)]
pop_dataset["label"] = [3 for _ in range(200)]
rock_dataset["label"] = [4 for _ in range(200)]
rap_dataset["label"] = [5 for _ in range(200)]

halk_dataset["sarki_sozu"] = pd.read_csv("../dataset/halk.csv", usecols=["sarki_sozu"])
sanat_dataset["sarki_sozu"] = pd.read_csv("../dataset/sanat.csv", usecols=["sarki_sozu"])
ilahi_dataset["sarki_sozu"] = pd.read_csv("../dataset/ilahi.csv", usecols=["sarki_sozu"])
pop_dataset["sarki_sozu"] = pd.read_csv("../dataset/pop.csv", usecols=["sarki_sozu"])
rock_dataset["sarki_sozu"] = pd.read_csv("../dataset/rock.csv", usecols=["sarki_sozu"])
rap_dataset["sarki_sozu"] = pd.read_csv("../dataset/rap.csv", usecols=["sarki_sozu"])

dataset = pd.concat([halk_dataset, sanat_dataset, ilahi_dataset, pop_dataset, rock_dataset, rap_dataset], axis=0, ignore_index=True)

dataset = dataset.reset_index(drop=True)

### Veri Önişleme

In [3]:
temp_lyrics = [lyrics.replace("\n\n", "\n") for lyrics in dataset.loc[:,"sarki_sozu"]]
dataset["sarki_sozu"] = temp_lyrics

temp_lyrics = []
for lyrics in dataset.loc[:,"sarki_sozu"]:
    lines = lyrics.split("\n")
    lyrics = ""
    for line in lines:
        lyrics += line.rstrip() + "\n"
    temp_lyrics.append(lyrics)
dataset["sarki_sozu"] = temp_lyrics

temp_lyrics = [lyrics.translate(str.maketrans('', '', string.punctuation)) for lyrics in dataset.loc[:,"sarki_sozu"]]
dataset["sarki_sozu"] = temp_lyrics

temp_lyrics = [lyrics.lower() for lyrics in dataset.loc[:,"sarki_sozu"]]
dataset["sarki_sozu"] = temp_lyrics


In [4]:
my_file = open("../raw_data/stop_words.txt", "r")
  
# reading the file
stop_words = my_file.read()
  
# replacing end of line('/n') with ' ' and
# splitting the text it further when '.' is seen.
stop_words = stop_words.replace('\n', ' ').split(" ")
  
# printing the data
my_file.close()

In [5]:
def remove_stopwords(text):
    # Split text into lines
    lines = text.split('\n')
    
    # Remove stopwords from each line
    for i, line in enumerate(lines):
        words = line.split()
        filtered_words = [word for word in words if word.lower() not in stop_words]
        lines[i] = ' '.join(filtered_words)
    
    # Join lines back together with line breaks
    return '\n'.join(lines)

temp_lyrics = []
for lyrics in dataset.loc[:,"sarki_sozu"]:
    filtered_text = remove_stopwords(lyrics)
    temp_lyrics.append(filtered_text)

dataset["sarki_sozu"] = temp_lyrics

In [6]:
temp_lyrics = [lyrics.replace("\n", "[SEP]").rstrip("[SEP]") for lyrics in dataset.loc[:,"sarki_sozu"]]
dataset["sarki_sozu"] = temp_lyrics

##### Load Fasttext Embeddings

In [6]:
path = '../models/cc.tr.300.bin'
if (not os.path.exists(path)):
    fasttext.util.download_model('tr', if_exists='ignore')
    os.replace("cc.tr.300.bin", "../models/cc.tr.300.bin")
    os.remove("cc.tr.300.bin.gz")

ft = fasttext.load_model(r"../models/cc.tr.300.bin")


 (100.00%) [==================================================>]                                                 ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]> 

In [7]:
ft.get_dimension()

300

In [25]:
split_chars = ["[CLS]", " ", "[SEP]"]

splitted_words_for_all_lyrics_in_data = []
for lyrics in dataset.loc[:, "sarki_sozu"]:
    words = []
    for char in split_chars:
        lyrics = lyrics.replace(char, " ")
    words = lyrics.split()
    splitted_words_for_all_lyrics_in_data.append(words)
kelimeler = splitted_words_for_all_lyrics_in_data

In [27]:
sent_list=[]
for lyrics in kelimeler:
    temp_sent_list = []
    for word in lyrics:
        wv=ft.get_word_vector(word)
        temp_sent_list.append(wv)
    sent_list.append(temp_sent_list)

import numpy as np 
sent_embed= np.mean(sent_list[0],axis=0)